Problem Statement.

There is a ball in a maze with empty spaces (represented as 0) and walls (represented as 1). The ball can go through the empty spaces by rolling up, down, left or right, but it won't stop rolling until hitting a wall. When the ball stops, it could choose the next direction. There is also a hole in this maze. The ball will drop into the hole if it rolls onto the hole.

Given the m x n maze, the ball's position ball and the hole's position hole, where ball = [ballrow, ballcol] and hole = [holerow, holecol], return a string instructions of all the instructions that the ball should follow to drop in the hole with the shortest distance possible. If there are multiple valid instructions, return the lexicographically minimum one. If the ball can't drop in the hole, return "impossible".

If there is a way for the ball to drop in the hole, the answer instructions should contain the characters 'u' (i.e., up), 'd' (i.e., down), 'l' (i.e., left), and 'r' (i.e., right).

The distance is the number of empty spaces traveled by the ball from the start position (excluded) to the destination (included).

You may assume that the borders of the maze are all walls (see examples).

 

Example 1:

Input: maze = [[0,0,0,0,0],[1,1,0,0,1],[0,0,0,0,0],[0,1,0,0,1],[0,1,0,0,0]], ball = [4,3], hole = [0,1]
Output: "lul"
Explanation: There are two shortest ways for the ball to drop into the hole.
The first way is left -> up -> left, represented by "lul".
The second way is up -> left, represented by 'ul'.
Both ways have shortest distance 6, but the first way is lexicographically smaller because 'l' < 'u'. So the output is "lul".

Example 2:

Input: maze = [[0,0,0,0,0],[1,1,0,0,1],[0,0,0,0,0],[0,1,0,0,1],[0,1,0,0,0]], ball = [4,3], hole = [3,0]
Output: "impossible"
Explanation: The ball cannot reach the hole.

Example 3:

Input: maze = [[0,0,0,0,0,0,0],[0,0,1,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,1]], ball = [0,4], hole = [3,5]
Output: "dldr"

 

Constraints:

    m == maze.length
    n == maze[i].length
    1 <= m, n <= 100
    maze[i][j] is 0 or 1.
    ball.length == 2
    hole.length == 2
    0 <= ballrow, holerow <= m
    0 <= ballcol, holecol <= n
    Both the ball and the hole exist in an empty space, and they will not be in the same position initially.
    The maze contains at least 2 empty spaces.

# BFS - Queue

In [2]:
from typing import List
from collections import deque

class Solution:
    def findShortestWay(self, maze: List[List[int]], ball: List[int], hole: List[int]) -> str:
        m, n = len(maze), len(maze[0])
        directions = {'d':[1, 0], 'l': [0, -1], 'r': [0, 1], 'u': [-1, 0]}
        
        queue = deque([(0, '', ball[0], ball[1])])
        visited = {tuple(ball): (0, '')}
        minDistance, minInstr = float('inf'), ''
        
        def check_smaller(d0, i0, d1, i1):
            if d1 < d0:
                return d1, i1
            elif d0 < d1:
                return d0, i0
            elif d1 == d0:
                lexicoSmaller = False
                if not i0: lexicoSmaller = True
                else:
                    for idx in range(min(len(i1), len(i0))):
                        if i1[idx] < i0[idx]:
                            lexicoSmaller = True
                            break
                        elif i1[idx] > i0[idx]:
                            break

                if lexicoSmaller: return d1, i1
                else: return d0, i0
        
        while queue:
            distance, instr, r, c = queue.popleft()
            
            for d_name, dir in  directions.items():
                r1, c1 = r + dir[0], c + dir[1]
                curr_d = distance
                while 0<= r1 < m and 0<= c1 < n and maze[r1][c1] == 0:
                    curr_d += 1
                    if [r1, c1] == hole:
                        minDistance, minInstr = check_smaller(minDistance, minInstr, curr_d, instr+d_name)
                                         
                    r1, c1 = r1 + dir[0], c1 + dir[1]
                    
                new_r, new_c = r1 - dir[0], c1 - dir[1]
                if (new_r, new_c) != (r,c) and ((new_r, new_c) not in visited or (curr_d, instr + d_name) == \
                      check_smaller(visited[(new_r, new_c)][0], visited[(new_r, new_c)][1], curr_d, instr+d_name)):
                    visited[(new_r, new_c)] = (curr_d, instr + d_name)
                    queue.append((curr_d, instr + d_name, new_r, new_c))
            
        return "impossible" if not minInstr else minInstr

# Djikstra's using heap and BFS - O(M * N) runtime, O(M * N) space

In [4]:
from typing import List
from heapq import heappush, heappop

class Solution:
    def findShortestWay(self, maze: List[List[int]], ball: List[int], hole: List[int]) -> str:
        m, n = len(maze), len(maze[0])
        directions = [['d', 1, 0], ['l', 0, -1], ['r', 0, 1], ['u', -1, 0]]
        
        minheap = []
        heappush(minheap, (0, ball[0], ball[1], ''))
        visited = set()
        
        while minheap:
            distance, r, c, instr = heappop(minheap)
            if [r, c] == hole: return instr
            if (r, c) not in visited:
                visited.add((r, c))

                for dir_name, dr, dc in directions:
                    r1, c1 = r, c
                    curr_d = distance

                    while 0 <= r1+dr < m and 0 <= c1+dc < n and maze[r1+dr][c1+dc] == 0:
                        r1, c1 = r1+dr, c1+dc
                        curr_d += 1
                        if [r1, c1] == hole:
                            break

                    if (r1, c1) not in visited:
                        heappush(minheap, (curr_d, r1, c1, instr+dir_name))
                    
        return 'impossible'

In [5]:
instance = Solution()
instance.findShortestWay([[0,0,0,0,0,0,0],[0,0,1,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,1]], [0,4], [3,5])

'dldr'